In [2]:
import pandas as pd

sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'    

csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df_googlesheet = pd.read_csv(csv_export_url)
df_googlesheet.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# If the S3 file is private, you will need your S3 configurations setup properly.
df_s3 = pd.read_csv('https://s3.amazonaws.com/irs-form-990/index_2011.csv')
df_s3.head()

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID
0,9091250,EFILE,591971002,201009,11/30/2011 1:06:39 AM,ANGELUS INC,990,93493316003251,201103169349300325
1,9091274,EFILE,251713602,201106,11/30/2011 1:09:14 AM,TOUCH-STONE SOLUTIONS INC,990,93493313012311,201113139349301231
2,9091275,EFILE,232705170,201012,11/30/2011 1:09:16 AM,RONALD MCDONALD HOUSE CHARITIES- PHILADELPHIA ...,990,93493313013011,201113139349301301
3,9091276,EFILE,581805618,201106,11/30/2011 1:09:19 AM,TORRINGTON VOA ELDERLY HOUSING INC BELL PARK T...,990,93493313013111,201113139349301311
4,9091277,EFILE,581876019,201106,11/30/2011 1:09:21 AM,HOUSTON VOA INDEPENDENT HOUSING INC HEIGHTS MANOR,990,93493313013161,201113139349301316


In [4]:
import env

def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

df = pd.read_sql('SELECT * FROM passengers', get_connection('titanic_db'))

df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


In [6]:
from aquire import get_titanic_data
df=get_titanic_data()

In [13]:
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


In [8]:
df.tail()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
886,886,0,2,male,27.0,0,0,13.00,S,Second,None,Southampton,1
887,887,1,1,female,19.0,0,0,30.00,S,First,B,Southampton,1
888,888,0,3,female,NaN,1,2,23.45,S,Third,None,Southampton,0
889,889,1,1,male,26.0,0,0,30.00,C,First,C,Cherbourg,1
890,890,0,3,male,32.0,0,0,7.75,Q,Third,None,Queenstown,1


In [9]:
df.shape

(891, 13)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
passenger_id    891 non-null int64
survived        891 non-null int64
pclass          891 non-null int64
sex             891 non-null object
age             714 non-null float64
sibsp           891 non-null int64
parch           891 non-null int64
fare            891 non-null float64
embarked        889 non-null object
class           891 non-null object
deck            203 non-null object
embark_town     889 non-null object
alone           891 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


In [11]:
df.describe()

,passenger_id,survived,pclass,age,sibsp,parch,fare,alone
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,445.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.602694
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,0.489615
min,0.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,222.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,445.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,1.000000
75%,667.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,890.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [12]:
df.sex.value_counts()

male      577
female    314
Name: sex, dtype: int64

In [14]:
df.fare.value_counts(bins=10, sort=False)

(-0.513, 51.233]      732
(51.233, 102.466]     106
(102.466, 153.699]     31
(153.699, 204.932]      2
(204.932, 256.165]     11
(256.165, 307.398]      6
(307.398, 358.63]       0
(358.63, 409.863]       0
(409.863, 461.096]      0
(461.096, 512.329]      3
Name: fare, dtype: int64

In [15]:
df.embark_town.value_counts(dropna=False)

Southampton    644
Cherbourg      168
Queenstown      77
NaN              2
Name: embark_town, dtype: int64

In [16]:
df.embark_town.fillna('Other', inplace=True)

In [17]:
df.embark_town.value_counts()

Southampton    644
Cherbourg      168
Queenstown      77
Other            2
Name: embark_town, dtype: int64

In [18]:
df.drop(columns=['deck'], inplace=True)

In [19]:
df.embarked.value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: embarked, dtype: int64

In [21]:
encoder=LabelEncoder()
df.embarked.fillna('Unknown',inplace=True)
encoder.fit(df.embarked)
df.embarked=encoder.transform(df.embarked)
df.embarked.head()

0    2
1    0
2    2
3    2
4    2
Name: embarked, dtype: int64

In [22]:
df.embarked.unique()

array([2, 0, 1, 3])

In [23]:
df.isnull().sum()

passenger_id      0
survived          0
pclass            0
sex               0
age             177
sibsp             0
parch             0
fare              0
embarked          0
class             0
embark_town       0
alone             0
dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=.30,random_state=123,stratify=df.survived)

In [35]:
scaler=MinMaxScaler()
scaler.fit(train[['fare']])
train.fare=scaler.transform(train[['fare']])

scaler=MinMaxScaler()
scaler.fit(train[['age']])
train.age=scaler.transform(train[['age']])

In [36]:
train[['age','fare']].head(10)

,age,fare
199,0.334089,0.025374
468,NaN,0.015078
198,NaN,0.015127
574,0.220742,0.015713
776,NaN,0.015127
148,0.511193,0.050749
227,0.284500,0.014151
408,0.291584,0.015176
762,0.277416,0.014110
166,NaN,0.107353


In [37]:
scaler = MinMaxScaler()
scaler.fit(test[['age', 'fare']])
test[['age', 'fare']] = scaler.transform(test[['age', 'fare']])

In [38]:
test[['age', 'fare']].head(10)

,age,fare
178,0.369721,0.049430
457,NaN,0.197196
16,0.016765,0.110741
95,NaN,0.030608
120,0.256271,0.279468
677,0.218455,0.037421
512,0.445355,0.099952
561,0.495777,0.030022
356,0.268877,0.209125
430,0.344510,0.100951
